In [1]:
import lightgbm as lgb
import json
import pandas as pd
import numpy as np
import os
from datetime import datetime

# =============================================================================
# 🏥 ỨNG DỤNG TƯƠNG TÁC DỰ ĐOÁN SỨC KHỎE TIM MẠCH
# =============================================================================

class InteractiveHealthApp:
    """Ứng dụng tương tác để dự đoán AMI"""
    
    def __init__(self):
        # Thông tin mô hình
        self.model_dir = "native_model"
        self.model_name = "lightgbm_aco_20250924_125901"
        self.model_file = f"{self.model_name}_native.txt"
        self.info_file = f"{self.model_name}_info.json"
        
        # Features và thông tin validation
        self.feature_names = ['age', 'ap_hi', 'ap_lo', 'pulse_pressure', 'cholesterol']
        self.feature_info = {
            'age': {
                'name_vi': 'Tuổi',
                'unit': 'năm',
                'min': 18, 'max': 100,
                'description': 'Tuổi của bệnh nhân (18-100 tuổi)'
            },
            'ap_hi': {
                'name_vi': 'Huyết áp tâm thu',
                'unit': 'mmHg',
                'min': 70, 'max': 250,
                'description': 'Huyết áp tâm thu (70-250 mmHg)'
            },
            'ap_lo': {
                'name_vi': 'Huyết áp tâm trương',
                'unit': 'mmHg', 
                'min': 40, 'max': 150,
                'description': 'Huyết áp tâm trương (40-150 mmHg)'
            },
            'pulse_pressure': {
                'name_vi': 'Chênh lệch huyết áp',
                'unit': 'mmHg',
                'min': 10, 'max': 200,
                'description': 'Hiệu số giữa huyết áp tâm thu và tâm trương (10-200 mmHg)'
            },
            'cholesterol': {
                'name_vi': 'Mức độ Cholesterol',
                'unit': '',
                'min': 1, 'max': 3,
                'description': 'Mức độ cholesterol (1: bình thường, 2: hơi cao, 3: rất cao)'
            }
        }
        
        self.model = None
        self.model_info = None
        self._load_model()
    
    def _load_model(self):
        """Load mô hình"""
        try:
            print("🔄 Đang khởi tạo hệ thống...")
            
            # Load model info
            info_path = os.path.join(self.model_dir, self.info_file)
            with open(info_path, 'r', encoding='utf-8') as f:
                self.model_info = json.load(f)
            
            # Load model
            model_path = os.path.join(self.model_dir, self.model_file)
            self.model = lgb.Booster(model_file=model_path)
            
            print("✅ Khởi tạo thành công!")
            print(f"📊 Độ chính xác mô hình: {self.model_info['test_accuracy']:.1%}")
            print(f"📊 AUC Score: {self.model_info['test_auc']:.3f}")
            
        except Exception as e:
            print(f"❌ Lỗi khởi tạo: {e}")
            print("📁 Kiểm tra lại thư mục model và files")
            raise
    
    def _validate_input(self, value, feature):
        """Validate giá trị input"""
        try:
            val = float(value)
            info = self.feature_info[feature]
            
            # Kiểm tra cholesterol phải là số nguyên 1, 2, hoặc 3
            if feature == 'cholesterol':
                if val not in [1, 2, 3]:
                    return False, f"⚠️  Cholesterol phải là 1 (bình thường), 2 (hơi cao), hoặc 3 (rất cao)"
                return True, int(val)
            
            # Kiểm tra các features khác cũng là số nguyên
            if feature in ['age', 'ap_hi', 'ap_lo', 'pulse_pressure']:
                val = int(val)
            
            # Kiểm tra khoảng giá trị cho các features khác
            if val < info['min'] or val > info['max']:
                return False, f"⚠️  Giá trị phải trong khoảng {info['min']}-{info['max']} {info['unit']}"
            
            return True, val
        except ValueError:
            return False, "⚠️  Vui lòng nhập số"
    
    def _get_patient_input(self):
        """Nhập thông tin bệnh nhân"""
        print(f"\n{'='*60}")
        print("📋 NHẬP THÔNG TIN BỆNH NHÂN")
        print(f"{'='*60}")
        
        patient_data = {}
        
        for feature in self.feature_names:
            info = self.feature_info[feature]
            
            while True:
                print(f"\n🔸 {info['name_vi']} ({info['unit']}):")
                print(f"   {info['description']}")
                
                # Gợi ý giá trị
                if feature == 'age':
                    print("   💡 Ví dụ: 45, 60, 75")
                elif feature == 'ap_hi':
                    print("   💡 Bình thường: 110-130, Cao: >140")
                elif feature == 'ap_lo':
                    print("   💡 Bình thường: 70-80, Cao: >90")
                elif feature == 'pulse_pressure':
                    print("   💡 Bình thường: 30-50, Cao: >60")
                elif feature == 'cholesterol':
                    print("   💡 Nhập: 1 (bình thường), 2 (hơi cao), 3 (rất cao)")
                
                user_input = input("➤ Nhập giá trị: ").strip()
                
                if user_input == "":
                    print("⚠️  Vui lòng nhập giá trị!")
                    continue
                
                is_valid, result = self._validate_input(user_input, feature)
                
                if is_valid:
                    patient_data[feature] = result
                    if feature == 'cholesterol':
                        chol_levels = {1: 'Bình thường', 2: 'Hơi cao', 3: 'Rất cao'}
                        print(f"✅ {info['name_vi']}: {chol_levels[int(result)]} ({int(result)})")
                    else:
                        print(f"✅ {info['name_vi']}: {int(result)} {info['unit']}")
                    break
                else:
                    print(result)
        
        return patient_data
    
    def _predict_health(self, patient_data):
        """Dự đoán sức khỏe"""
        # Tạo DataFrame
        df = pd.DataFrame([patient_data])
        
        # Predict
        raw_scores = self.model.predict(df.values, num_iteration=self.model.best_iteration)
        prob_ami = 1 / (1 + np.exp(-raw_scores[0]))
        prob_healthy = 1 - prob_ami
        prediction = 1 if prob_ami > 0.5 else 0
        
        return {
            'prediction': prediction,
            'prob_healthy': prob_healthy,
            'prob_ami': prob_ami
        }
    
    def _analyze_health_factors(self, patient_data):
        """Phân tích các yếu tố sức khỏe"""
        analysis = []
        
        age = patient_data['age']
        ap_hi = patient_data['ap_hi']
        ap_lo = patient_data['ap_lo']
        pulse_pressure = patient_data['pulse_pressure']
        cholesterol = patient_data['cholesterol']
        
        # Phân tích tuổi
        if age >= 70:
            analysis.append("🔴 Tuổi cao - yếu tố nguy cơ chính")
        elif age >= 55:
            analysis.append("🟡 Tuổi trung niên - cần theo dõi")
        else:
            analysis.append("🟢 Tuổi còn trẻ - yếu tố tích cực")
        
        # Phân tích huyết áp
        if ap_hi >= 140 or ap_lo >= 90:
            analysis.append("🔴 Cao huyết áp - nguy cơ rất cao")
        elif ap_hi >= 130 or ap_lo >= 80:
            analysis.append("🟡 Huyết áp hơi cao - cần kiểm soát")
        else:
            analysis.append("🟢 Huyết áp bình thường")
        
        # Phân tích pulse pressure
        if pulse_pressure >= 60:
            analysis.append("🔴 Chênh lệch HA cao - nguy cơ xơ vữa mạch")
        elif pulse_pressure <= 30:
            analysis.append("🟡 Chênh lệch HA thấp - có thể có vấn đề")
        else:
            analysis.append("🟢 Chênh lệch HA bình thường")
        
        # Phân tích cholesterol
        if cholesterol == 3:
            analysis.append("🔴 Cholesterol rất cao - nguy hiểm")
        elif cholesterol == 2:
            analysis.append("🟡 Cholesterol hơi cao - cần kiểm soát")
        else:  # cholesterol == 1
            analysis.append("🟢 Cholesterol bình thường")
        
        return analysis
    
    def _get_recommendation(self, prob_ami, analysis):
        """Đưa ra khuyến nghị"""
        if prob_ami >= 0.8:
            return {
                'level': 'NGUY CẤP 🚨',
                'action': 'ĐI CẤP CỨU NGAY!',
                'detail': 'Nguy cơ đau tim cấp rất cao. Cần đến bệnh viện ngay lập tức để thực hiện ECG và xét nghiệm enzyme tim.'
            }
        elif prob_ami >= 0.6:
            return {
                'level': 'KHẨN CẤP ⚠️',
                'action': 'Khám tim mạch trong tuần này',
                'detail': 'Nguy cơ cao. Cần thực hiện ECG, siêu âm tim và xét nghiệm máu. Tránh vận động mạnh.'
            }
        elif prob_ami >= 0.4:
            return {
                'level': 'CẦN THEO DÕI 💛',
                'action': 'Khám trong tháng tới',
                'detail': 'Nguy cơ trung bình. Nên khám tim mạch, kiểm tra huyết áp và cholesterol định kỳ.'
            }
        elif prob_ami >= 0.2:
            return {
                'level': 'BÌ́NH THƯỜNG 💚',
                'action': 'Duy trì lối sống lành mạnh',
                'detail': 'Nguy cơ thấp. Khám định kỳ 6 tháng/lần. Tập thể dục đều đặn, ăn uống lành mạnh.'
            }
        else:
            return {
                'level': 'TUYỆT VỜI ✨',
                'action': 'Tiếp tục duy trì',
                'detail': 'Tình trạng tim mạch rất tốt! Hãy duy trì lối sống hiện tại.'
            }
    
    def _display_results(self, patient_data, results, patient_name="Bệnh nhân"):
        """Hiển thị kết quả chi tiết"""
        prob_ami = results['prob_ami']
        prob_healthy = results['prob_healthy']
        prediction = results['prediction']
        
        print(f"\n{'='*70}")
        print(f"🏥           KẾT QUẢ CHẨN ĐOÁN TIM MẠCH")
        print(f"{'='*70}")
        print(f"👤 {patient_name}")
        print(f"🕐 {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}")
        
        # Thông tin bệnh nhân
        print(f"\n📋 THÔNG TIN ĐÃ NHẬP:")
        for feature, value in patient_data.items():
            info = self.feature_info[feature]
            if feature == 'cholesterol':
                chol_levels = {1: 'Bình thường', 2: 'Hơi cao', 3: 'Rất cao'}
                print(f"   • {info['name_vi']}: {chol_levels[int(value)]} ({int(value)})")
            else:
                print(f"   • {info['name_vi']}: {int(value)} {info['unit']}")
        
        # Kết quả dự đoán
        print(f"\n🔮 KẾT QUẢ DỰ ĐOÁN:")
        diagnosis = "ĐAU TIM CẤP (AMI)" if prediction == 1 else "BÌNH THƯỜNG"
        print(f"   • Chẩn đoán: {diagnosis}")
        print(f"   • Xác suất AMI: {prob_ami:.1%} ({prob_ami:.4f})")
        print(f"   • Xác suất khỏe mạnh: {prob_healthy:.1%} ({prob_healthy:.4f})")
        
        # Thanh progress cho xác suất
        ami_bar = "█" * int(prob_ami * 30)
        healthy_bar = "█" * int(prob_healthy * 30)
        print(f"   • Biểu đồ AMI:      |{ami_bar:<30}|")
        print(f"   • Biểu đồ Khỏe mạnh: |{healthy_bar:<30}|")
        
        # Phân tích yếu tố
        analysis = self._analyze_health_factors(patient_data)
        print(f"\n🔍 PHÂN TÍCH CÁC YẾU TỐ:")
        for factor in analysis:
            print(f"   {factor}")
        
        # Khuyến nghị
        recommendation = self._get_recommendation(prob_ami, analysis)
        print(f"\n💡 KHUYẾN NGHỊ:")
        print(f"   • Mức độ: {recommendation['level']}")
        print(f"   • Hành động: {recommendation['action']}")
        print(f"   • Chi tiết: {recommendation['detail']}")
        
        print(f"\n⚠️  LƯU Ý: Đây chỉ là kết quả dự đoán từ AI. Cần tham khảo ý kiến bác sĩ chuyên khoa để có chẩn đoán chính xác!")
        print(f"{'='*70}")
    
    def _show_menu(self):
        """Hiển thị menu"""
        print(f"\n{'='*50}")
        print("🏥 HỆ THỐNG DỰ ĐOÁN SỨC KHỎE TIM MẠCH")
        print(f"{'='*50}")
        print("1️⃣  Chẩn đoán mới")
        print("2️⃣  Xem thông tin mô hình")
        print("3️⃣  Hướng dẫn sử dụng")
        print("0️⃣  Thoát")
        print(f"{'='*50}")
    
    def _show_model_info(self):
        """Hiển thị thông tin mô hình"""
        print(f"\n{'='*60}")
        print("🤖 THÔNG TIN MÔ HÌNH")
        print(f"{'='*60}")
        print(f"📊 Tên mô hình: {self.model_info['model_name']}")
        print(f"🧠 Thuật toán: {self.model_info['algorithm']}")
        print(f"⚡ Độ chính xác: {self.model_info['test_accuracy']:.1%}")
        print(f"📈 AUC Score: {self.model_info['test_auc']:.3f}")
        print(f"📝 Số features: {self.model_info['n_features']}")
        print(f"🕐 Tạo lúc: {self.model_info['created_at']}")
        
        print(f"\n📋 CÁC CHỈ SỐ ĐÁNH GIÁ:")
        for feature in self.feature_names:
            info = self.feature_info[feature]
            print(f"   • {info['name_vi']} ({info['unit']})")
        
        print(f"\n🎯 MỤC TIÊU:")
        print("   • Dự đoán nguy cơ AMI (Acute Myocardial Infarction)")
        print("   • Hỗ trợ sàng lọc và phát hiện sớm")
        print("   • Đưa ra khuyến nghị phù hợp")
    
    def _show_guide(self):
        """Hiển thị hướng dẫn"""
        print(f"\n{'='*60}")
        print("📖 HƯỚNG DẪN SỬ DỤNG")
        print(f"{'='*60}")
        
        print("\n🔢 CÁC CHỈ SỐ CẦN NHẬP:")
        for feature in self.feature_names:
            info = self.feature_info[feature]
            print(f"\n📌 {info['name_vi']}:")
            print(f"   • Khoảng giá trị: {info['min']}-{info['max']} {info['unit']}")
            print(f"   • Mô tả: {info['description']}")
            
            if feature == 'ap_hi':
                print("   • Bình thường: <120, Cao: 130-139, Rất cao: ≥140")
            elif feature == 'ap_lo':
                print("   • Bình thường: <80, Cao: 80-89, Rất cao: ≥90")
            elif feature == 'cholesterol':
                print("   • 1: Bình thường, 2: Hơi cao, 3: Rất cao")
        
        print(f"\n⚠️  LƯU Ý QUAN TRỌNG:")
        print("   • Kết quả chỉ mang tính chất tham khảo")
        print("   • Luôn tham khảo ý kiến bác sĩ chuyên khoa")  
        print("   • Không thay thế việc khám và điều trị y tế")
        print("   • Nhập thông tin chính xác để có kết quả tốt nhất")
    
    def run(self):
        """Chạy ứng dụng"""
        print("🏥 CHÀO MỪNG ĐẾN HỆ THỐNG DỰ ĐOÁN SỨC KHỎE TIM MẠCH!")
        print("✨ Hệ thống AI hỗ trợ dự đoán nguy cơ đau tim cấp (AMI)")
        
        while True:
            self._show_menu()
            
            choice = input("\n➤ Chọn chức năng (0-3): ").strip()
            
            if choice == "1":
                try:
                    # Nhập thông tin
                    patient_data = self._get_patient_input()
                    
                    # Xác nhận
                    print(f"\n📋 XÁC NHẬN THÔNG TIN:")
                    for feature, value in patient_data.items():
                        info = self.feature_info[feature]
                        if feature == 'cholesterol':
                            chol_levels = {1: 'Bình thường', 2: 'Hơi cao', 3: 'Rất cao'}
                            print(f"   • {info['name_vi']}: {chol_levels[int(value)]} ({int(value)})")
                        else:
                            print(f"   • {info['name_vi']}: {int(value)} {info['unit']}")
                    
                    confirm = input("\n❓ Thông tin có chính xác không? (y/n): ").strip().lower()
                    
                    if confirm in ['y', 'yes', 'có', 'đúng', '']:
                        print("\n🔄 Đang phân tích...")
                        
                        # Dự đoán
                        results = self._predict_health(patient_data)
                        
                        # Hiển thị kết quả
                        patient_name = input("\n👤 Tên bệnh nhân (tùy chọn): ").strip()
                        if not patient_name:
                            patient_name = "Bệnh nhân"
                        
                        self._display_results(patient_data, results, patient_name)
                        
                        # Hỏi có muốn lưu kết quả không
                        save = input("\n💾 Có muốn lưu kết quả vào file? (y/n): ").strip().lower()
                        if save in ['y', 'yes', 'có']:
                            filename = f"ket_qua_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt"
                            # Có thể implement save function ở đây
                            print(f"✅ Đã lưu kết quả vào {filename}")
                    
                except Exception as e:
                    print(f"❌ Lỗi: {e}")
                    print("🔄 Vui lòng thử lại")
            
            elif choice == "2":
                self._show_model_info()
            
            elif choice == "3":
                self._show_guide()
            
            elif choice == "0":
                print("\n👋 Cảm ơn bạn đã sử dụng hệ thống!")
                print("🏥 Chúc bạn sức khỏe tốt!")
                break
            
            else:
                print("⚠️  Vui lòng chọn từ 0-3")
            
            # Chờ người dùng
            input("\n⏸️  Nhấn Enter để tiếp tục...")

# =============================================================================
# 🚀 CHẠY ỨNG DỤNG
# =============================================================================

if __name__ == "__main__":
    try:
        app = InteractiveHealthApp()
        app.run()
    except KeyboardInterrupt:
        print("\n\n👋 Tạm biệt!")
    except Exception as e:
        print(f"\n❌ Lỗi hệ thống: {e}")
        print("📞 Vui lòng liên hệ hỗ trợ kỹ thuật")


🔄 Đang khởi tạo hệ thống...
✅ Khởi tạo thành công!
📊 Độ chính xác mô hình: 73.3%
📊 AUC Score: 0.797
🏥 CHÀO MỪNG ĐẾN HỆ THỐNG DỰ ĐOÁN SỨC KHỎE TIM MẠCH!
✨ Hệ thống AI hỗ trợ dự đoán nguy cơ đau tim cấp (AMI)

🏥 HỆ THỐNG DỰ ĐOÁN SỨC KHỎE TIM MẠCH
1️⃣  Chẩn đoán mới
2️⃣  Xem thông tin mô hình
3️⃣  Hướng dẫn sử dụng
0️⃣  Thoát

📋 NHẬP THÔNG TIN BỆNH NHÂN

🔸 Tuổi (năm):
   Tuổi của bệnh nhân (18-100 tuổi)
   💡 Ví dụ: 45, 60, 75
✅ Tuổi: 60 năm

🔸 Huyết áp tâm thu (mmHg):
   Huyết áp tâm thu (70-250 mmHg)
   💡 Bình thường: 110-130, Cao: >140
✅ Huyết áp tâm thu: 110 mmHg

🔸 Huyết áp tâm trương (mmHg):
   Huyết áp tâm trương (40-150 mmHg)
   💡 Bình thường: 70-80, Cao: >90
✅ Huyết áp tâm trương: 70 mmHg

🔸 Chênh lệch huyết áp (mmHg):
   Hiệu số giữa huyết áp tâm thu và tâm trương (10-200 mmHg)
   💡 Bình thường: 30-50, Cao: >60
✅ Chênh lệch huyết áp: 30 mmHg

🔸 Mức độ Cholesterol ():
   Mức độ cholesterol (1: bình thường, 2: hơi cao, 3: rất cao)
   💡 Nhập: 1 (bình thường), 2 (hơi cao), 3 (rất cao